In [1]:
import logging
from pprint import pprint
from resimulate.euicc.card import Card
from resimulate.euicc.exceptions import IccidAlreadyExists
from resimulate.euicc.models.activation_profile import ActivationProfile
from resimulate.euicc.models.reset_option import ResetOption
from resimulate.euicc.mutation.deterministic_engine import DeterministicMutationEngine
from resimulate.euicc.recorder.recorder import OperationRecorder
from resimulate.euicc.transport.pcsc_link import PcscLink

logging.basicConfig(level=logging.DEBUG)

mutation_engine = DeterministicMutationEngine()
recorder = OperationRecorder()

profile = ActivationProfile.from_activation_code(
    "LPA:1$rsp.truphone.com$QR-G-5C-1LS-1W1Z9P7"
)

print(profile)

with PcscLink(recorder=recorder, apdu_data_size=255) as link:
    card = Card(link)
    print([app.name for app in card.supported_applications.values()])
    try:
        print(card.isd_r.get_euicc_challenge())
    except Exception:
        pass
    print(card.isd_r.get_euicc_info_2())
    print(card.isd_r.get_euicc_info_1())
    print(card.isd_r.get_eid())
    """ try:
        notification = card.isd_r.download_profile(profile)
        iccid = notification.data.notification.iccid
        pprint(notification)
        card.isd_r.process_notifications([notification])
        card.isd_r.set_nickname(iccid, "Some Nickname")
        card.isd_r.enable_profile(iccid)
        card.isd_r.disable_profile(iccid)
    except IccidAlreadyExists:
        pprint("Profile already exists")
        pass """

    profiles = card.isd_r.get_profiles()
    pprint(profiles)
    for profile in profiles:
        # card.isd_r.delete_profile(isdp_aid=profile.isdp_aid)
        notifications = card.isd_r.retrieve_notification_list()
        card.isd_r.process_notifications(notifications)

    card.isd_r.set_default_dp_address("some random address")
    card.isd_r.reset_euicc_memory(ResetOption.RESET_DEFAULT_SMDP_ADDRESS)
    pprint(card.isd_r.get_profiles())

DEBUG:root:Cleared all recorded operations
DEBUG:root:Initializing recorder...
DEBUG:root:Disconnected from device HID Global OMNIKEY 3x21 Smart Card Reader [OMNIKEY 3x21 Smart Card Reader] 00 00
DEBUG:root:Connecting to device HID Global OMNIKEY 3x21 Smart Card Reader [OMNIKEY 3x21 Smart Card Reader] 00 00 using protocol T1
DEBUG:root:Connected to device HID Global OMNIKEY 3x21 Smart Card Reader [OMNIKEY 3x21 Smart Card Reader] 00 00
DEBUG:root:Disconnected from device HID Global OMNIKEY 3x21 Smart Card Reader [OMNIKEY 3x21 Smart Card Reader] 00 00
DEBUG:root:Disconnected from device HID Global OMNIKEY 3x21 Smart Card Reader [OMNIKEY 3x21 Smart Card Reader] 00 00
DEBUG:root:Connecting to device HID Global OMNIKEY 3x21 Smart Card Reader [OMNIKEY 3x21 Smart Card Reader] 00 00 using protocol T1


ActivationProfile(smdp_address='rsp.truphone.com', matching_id='QR-G-5C-1LS-1W1Z9P7', confirmation_code=None)


DEBUG:root:Connected to device HID Global OMNIKEY 3x21 Smart Card Reader [OMNIKEY 3x21 Smart Card Reader] 00 00
DEBUG:root:Sending TPDU: 01A4040C07A0000000871002
DEBUG:root:Received Data: None, SW: 9000
DEBUG:root:Selected ADF a0000000871002
DEBUG:root:Found USIM via ADF a0000000871002
DEBUG:root:Sending TPDU: 01A4040C10A0000005591010FFFFFFFF8900000100
DEBUG:root:Received Data: None, SW: 6a82
DEBUG:root:Error selecting ADF A0000005591010FFFFFFFF8900000100! Failed with SW match failed! Expected 9000 and got 6a82.
DEBUG:root:Sending TPDU: 00A4040C10A0000005591010FFFFFFFF8900000100
DEBUG:root:Received Data: None, SW: 6a82
DEBUG:root:Error selecting ADF A0000005591010FFFFFFFF8900000100! Failed with SW match failed! Expected 9000 and got 6a82.
DEBUG:root:Sending TPDU: 01A4040C10A0000005591010FFFFFFFF8900050500
DEBUG:root:Received Data: None, SW: 6a82
DEBUG:root:Error selecting ADF A0000005591010FFFFFFFF8900050500! Failed with SW match failed! Expected 9000 and got 6a82.
DEBUG:root:Sending T

['USIM', 'ISDR', 'ECASD']
17b5cf2995a2bef2d3c7ed12654fcbeb
base_profile_package_version='2.3.1' lowest_svn='2.2.2' euicc_firmware_version='4.2.0' ext_card_resource='81010082040005f69c830209e6' uicc_capability=<UICCCapability flags=['USIM_SUPPORT', 'ISIM_SUPPORT', 'CSIM_SUPPORT', 'AKA_MILENAGE', 'AKA_CAVE', 'AKA_TUAK128', 'AKA_TUAK256', 'GBA_AUTHEN_USIM', 'GBA_AUTHEN_ISIM', 'MBMS_AUTHEN_USIM', 'EAP_CLIENT', 'JAVACARD', 'BER_TLV_FILE_SUPPORT', 'DF_LINK_SUPPORT', 'CAT_TP', 'GET_IDENTITY', 'PROFILE_A_X25519', 'PROFILE_B_P256']> ts102241_version='9.2.0' globalplatform_version='2.3.0' euicc_rsp_capability=<EuiccRspCapability flags=['ADDITIONAL_PROFILE', 'TEST_PROFILE_SUPPORT']> euicc_ci_pkid_list_for_verification=['c1d4af8957f48cd51a47fef73123a04011d3d69e', '81370f5125d0b1d408d4c3b232e6d25e795bebfb'] euicc_ci_pkid_list_for_signing=['c1d4af8957f48cd51a47fef73123a04011d3d69e', '81370f5125d0b1d408d4c3b232e6d25e795bebfb'] euicc_category=<EuiccCategory.OTHER: 0> forbidden_profile_policy_rules=<Pp

DEBUG:root:Sending TPDU: 80C0000005
DEBUG:root:Received Data: bf2b02a000, SW: 9000
DEBUG:root:RetrieveNotificationsListResponse: ('notificationList', [])
DEBUG:root:Sending request: SetDefaultDpAddressRequest
DEBUG:root:Sending APDU (cla=80 ins=e2 p1=91 p2=00 lc=18 data=bf3f158013736f6d652072616e646f6d2061646472657373 le=00 extended=False)
DEBUG:root:Sending TPDU: 80E2910018BF3F158013736F6D652072616E646F6D2061646472657373
DEBUG:root:Received Data: None, SW: 6106
DEBUG:root:Sending TPDU: 80C0000006
DEBUG:root:Received Data: bf3f03800100, SW: 9000
DEBUG:root:SetDefaultDpAddressResponse: {'setDefaultDpAddressResult': 0}
DEBUG:root:Sending request: EuiccMemoryResetRequest
DEBUG:root:Sending APDU (cla=80 ins=e2 p1=91 p2=00 lc=07 data=bf340482020020 le=00 extended=False)
DEBUG:root:Sending TPDU: 80E2910007BF340482020020
DEBUG:root:Received Data: None, SW: 6106
DEBUG:root:Sending TPDU: 80C0000006
DEBUG:root:Received Data: bf3403800100, SW: 9000
DEBUG:root:ResetEuiccMemoryResponse: {'resetResu

[Profile(iccid='984474560000913341f7', isdp_aid='a0000005591010ffffffff8900001200', profile_state=<ProfileState.DISABLED: 0>, nickname=None, service_provider_name='BetterRoaming', profile_name='BetterRoaming', icon_type=None, icon=None, profile_class=<ProfileClass.PRODUCTION: 2>, notification_configuration_info=None, profile_owner=None, dp_proprietary_data=None, profile_policy_rules=None, service_specific_data_stored_in_euicc=None, rpm_configuration=None, hri_server_address=None, lpr_configuration=None, enterprise_configuration=None, service_description=None, device_change_configuration=None, enabled_on_esim_port=None, profile_size=None)]


In [1]:
import logging
from pprint import pprint
from resimulate.euicc.card import Card
from resimulate.euicc.exceptions import IccidAlreadyExists
from resimulate.euicc.models.activation_profile import ActivationProfile
from resimulate.euicc.models.reset_option import ResetOption
from resimulate.euicc.mutation.deterministic_engine import DeterministicMutationEngine
from resimulate.euicc.recorder.recorder import OperationRecorder
from resimulate.euicc.transport.pcsc_link import PcscLink

logging.basicConfig(level=logging.DEBUG)

profile = ActivationProfile.from_activation_code(
    "LPA:1$rsp.truphone.com$QR-G-5C-1LS-1W1Z9P7"
)

print(profile)

with PcscLink(apdu_data_size=255) as link:
    card = Card(link)
    print(card.isd_r.get_euicc_challenge())

DEBUG:root:Disconnected from device HID Global OMNIKEY 3x21 Smart Card Reader [OMNIKEY 3x21 Smart Card Reader] 00 00
DEBUG:root:Connecting to device HID Global OMNIKEY 3x21 Smart Card Reader [OMNIKEY 3x21 Smart Card Reader] 00 00 using protocol T1
DEBUG:root:Connected to device HID Global OMNIKEY 3x21 Smart Card Reader [OMNIKEY 3x21 Smart Card Reader] 00 00
DEBUG:root:Sending TPDU: 01A4040C07A0000000871002


ActivationProfile(smdp_address='rsp.truphone.com', matching_id='QR-G-5C-1LS-1W1Z9P7', confirmation_code=None)


DEBUG:root:Received Data: None, SW: 9000
DEBUG:root:Selected ADF a0000000871002
DEBUG:root:Found USIM via ADF a0000000871002
DEBUG:root:Sending TPDU: 01A4040C10A0000005591010FFFFFFFF8900000100
DEBUG:root:Received Data: None, SW: 6121
DEBUG:root:Sending TPDU: 01C0000021
DEBUG:root:Received Data: 6f1f8410a0000005591010ffffffff8900000100a5049f6501ffe0058203020300, SW: 9000
DEBUG:root:Selected ADF A0000005591010FFFFFFFF8900000100
DEBUG:root:Found ISDR via ADF A0000005591010FFFFFFFF8900000100
DEBUG:root:Sending TPDU: 01A4040C10A0000005591010FFFFFFFF8900000200
DEBUG:root:Received Data: None, SW: 611a
DEBUG:root:Sending TPDU: 01C000001A
DEBUG:root:Received Data: 6f188410a0000005591010ffffffff8900000200a5049f6501ff, SW: 9000
DEBUG:root:Selected ADF A0000005591010FFFFFFFF8900000200
DEBUG:root:Found ECASD via ADF A0000005591010FFFFFFFF8900000200
DEBUG:root:Sending TPDU: 01A4040C10A0000005591010FFFFFFFF8900000300
DEBUG:root:Received Data: None, SW: 6a82
DEBUG:root:Error selecting ADF A00000055910

741c29b1f3ca35a436f20d41211310a2


In [2]:
import logging

""" 
logging.basicConfig(level=logging.INFO) """

from resimulate.euicc.mutation.deterministic_engine import DeterministicMutationEngine
from resimulate.fuzzing.apdu_fuzzing.models.scenario_runner import ScenarioRunner
from resimulate.fuzzing.apdu_fuzzing import SCENARIOS, ProfileDownloadScenario

runner = ScenarioRunner(scenarios=[ProfileDownloadScenario])
runner.record_card(
    "9esimv2",
    path="recordings/",
    overwrite=True,
    mutation_engine=DeterministicMutationEngine(),
)

 root 
 root 
   └── get_euicc_challenge (random_byte, ApduException)
 root 
   ├── get_euicc_challenge (random_byte, ApduException)
   └── get_euicc_challenge (truncate, ApduException)
 root 
   ├── get_euicc_challenge (random_byte, ApduException)
   ├── get_euicc_challenge (truncate, ApduException)
   └── get_euicc_challenge (none, success)
 root 
   ├── get_euicc_challenge (random_byte, ApduException)
   ├── get_euicc_challenge (truncate, ApduException)
   └── get_euicc_challenge (none, success)
      └── get_euicc_info_1 (random_byte, ApduException)
 root 
   ├── get_euicc_challenge (random_byte, ApduException)
   ├── get_euicc_challenge (truncate, ApduException)
   ├── get_euicc_challenge (none, success)
   │  └── get_euicc_info_1 (random_byte, ApduException)
   └── get_euicc_challenge (zero_block, ApduException)
 root 
   ├── get_euicc_challenge (random_byte, ApduException)
   ├── get_euicc_challenge (truncate, ApduException)
   ├── get_euicc_challenge (none, success)
   │  └── g

KeyboardInterrupt: 

In [2]:
import logging
from pprint import pprint
from resimulate.euicc.card import Card
from resimulate.euicc.exceptions import IccidAlreadyExists
from resimulate.euicc.models.activation_profile import ActivationProfile
from resimulate.euicc.models.reset_option import ResetOption
from resimulate.euicc.mutation.deterministic_engine import DeterministicMutationEngine
from resimulate.euicc.recorder.recorder import OperationRecorder
from resimulate.euicc.transport.apdu import APDUPacket
from resimulate.euicc.transport.pcsc_link import PcscLink

logging.basicConfig(level=logging.DEBUG)

with PcscLink(apdu_data_size=255) as link:
    card = Card(link)
    # select ef.dir with data 2F00
    select = APDUPacket(cla=0x00, ins=0xA4, p1=0x00, p2=0x0C, data=b"\x2f\x00")
    # read binary
    read_binary = APDUPacket(cla=0x00, ins=0xB0, p1=0x00, p2=0x00, le=0x00)
    data, sw = link.send_apdu_with_mutation("isd_r", "test", select)
    pprint(data)
    data, sw = link.send_apdu_with_mutation("isd_r", "test", read_binary)
    pprint(data)

DEBUG:root:Disconnected from device HID Global OMNIKEY 3x21 Smart Card Reader [OMNIKEY 3x21 Smart Card Reader] 00 00
DEBUG:root:Connecting to device HID Global OMNIKEY 3x21 Smart Card Reader [OMNIKEY 3x21 Smart Card Reader] 00 00 using protocol T1
DEBUG:root:Connected to device HID Global OMNIKEY 3x21 Smart Card Reader [OMNIKEY 3x21 Smart Card Reader] 00 00
DEBUG:root:Sent TPDU: 80a4040407a0000000871002, received data: , SW: 6a82
DEBUG:root:Error selecting ADF a0000000871002! Failed with SW match failed! Expected 9000 and got 6a82.
DEBUG:root:Sent TPDU: 00a4040407a0000000871002, received data: , SW: 6138
DEBUG:root:Sent TPDU: 00c0000038, received data: 62368202782183027ff08410a0000000871002ffffffff89030500018a01058b032f060cc60c90016083010183018183010a8104000621ff, SW: 9000
DEBUG:root:Selected ADF a0000000871002
DEBUG:root:Found USIM via ADF a0000000871002 with CLA byte 00
DEBUG:root:Sent TPDU: 80a4040410a0000005591010ffffffff8900000100, received data: , SW: 6a82
DEBUG:root:Error select

''
None


In [4]:
import logging
from pprint import pprint
from resimulate.euicc.card import Card
from resimulate.euicc.models.reset_option import ResetOption
from resimulate.euicc.transport.pcsc_link import PcscLink

logging.basicConfig(level=logging.DEBUG)

with PcscLink(apdu_data_size=255) as link:
    card = Card(link)
    card.isd_r.reset_euicc_memory(ResetOption.all())
    pprint(card.isd_r.get_profiles())
    pprint(card.isd_r.get_notifications())

DEBUG:root:Disconnected from device HID Global OMNIKEY 3x21 Smart Card Reader [OMNIKEY 3x21 Smart Card Reader] 00 00


DEBUG:root:Connecting to device HID Global OMNIKEY 3x21 Smart Card Reader [OMNIKEY 3x21 Smart Card Reader] 00 00 using protocol T1
DEBUG:root:Connected to device HID Global OMNIKEY 3x21 Smart Card Reader [OMNIKEY 3x21 Smart Card Reader] 00 00
DEBUG:root:Sending TPDU: 01A4040C07A0000000871002
DEBUG:root:Received Data: None, SW: 9000
DEBUG:root:Selected ADF a0000000871002
DEBUG:root:Found USIM via ADF a0000000871002
DEBUG:root:Sending TPDU: 01A4040C10A0000005591010FFFFFFFF8900000100
DEBUG:root:Received Data: None, SW: 6a82
DEBUG:root:Error selecting ADF A0000005591010FFFFFFFF8900000100! Failed with SW match failed! Expected 9000 and got 6a82.
DEBUG:root:Sending TPDU: 00A4040C10A0000005591010FFFFFFFF8900000100
DEBUG:root:Received Data: None, SW: 6a82
DEBUG:root:Error selecting ADF A0000005591010FFFFFFFF8900000100! Failed with SW match failed! Expected 9000 and got 6a82.
DEBUG:root:Sending TPDU: 01A4040C10A0000005591010FFFFFFFF8900050500
DEBUG:root:Received Data: None, SW: 6a82
DEBUG:root:

[]
[Notification(seq_number=148, event=<NotificationEvent flags=['INSTALL']>, address='rsp.truphone.com', iccid='984474560000913392f5'),
 Notification(seq_number=149, event=<NotificationEvent flags=['INSTALL']>, address='rsp.truphone.com', iccid='984474560000913392f5'),
 Notification(seq_number=150, event=<NotificationEvent flags=['LOCAL_DELETE']>, address='rsp.truphone.com', iccid='984474560000913341f7'),
 Notification(seq_number=151, event=<NotificationEvent flags=['LOCAL_DELETE']>, address='rsp.truphone.com', iccid='984474560000913392f5'),
 Notification(seq_number=152, event=<NotificationEvent flags=['INSTALL']>, address='rsp.truphone.com', iccid='984474560000913392f5'),
 Notification(seq_number=153, event=<NotificationEvent flags=['LOCAL_DELETE']>, address='rsp.truphone.com', iccid='984474560000913392f5'),
 Notification(seq_number=154, event=<NotificationEvent flags=['INSTALL']>, address='rsp.truphone.com', iccid='984474560000913341f7'),
 Notification(seq_number=155, event=<Notific

In [4]:
reset_options = [
    ResetOption.RESET_DEFAULT_SMDP_ADDRESS,
    ResetOption.DELETE_PROVISIONING_PROFILES,
]

reset_value = 0
for option in reset_options:
    reset_value |= 1 << option.value

bin(reset_value)[2:].encode()

b'10100'

In [1]:
import logging
from pprint import pprint
from resimulate.euicc.card import Card
from resimulate.euicc.models.reset_option import ResetOption
from resimulate.euicc.transport.pcsc_link import PcscLink
from resimulate.util import str2ascii

logging.basicConfig(level=logging.DEBUG)
BIN_PATH = "/home/niklas/Downloads/ESTKme-2024-3.4.3/files/T001V06-3.4.3"

with PcscLink(apdu_data_size=255) as link:
    card = Card(link)
    print([app.name for app in card.supported_applications.values()])
    print(card.estk_dfu.get_version())
    pprint(data)

DEBUG:root:Disconnected from device HID Global OMNIKEY 3x21 Smart Card Reader [OMNIKEY 3x21 Smart Card Reader] 00 00
DEBUG:root:Connecting to device HID Global OMNIKEY 3x21 Smart Card Reader [OMNIKEY 3x21 Smart Card Reader] 00 00 using protocol T1
DEBUG:root:Connected to device HID Global OMNIKEY 3x21 Smart Card Reader [OMNIKEY 3x21 Smart Card Reader] 00 00
DEBUG:root:Sending TPDU: 01A4040C07A0000000871002
DEBUG:root:Received Data: None, SW: 6999
DEBUG:root:Error selecting ADF a0000000871002! Failed with SW match failed! Expected 9000 and got 6999.
DEBUG:root:Sending TPDU: 00A4040C07A0000000871002
DEBUG:root:Received Data: None, SW: 6a82
DEBUG:root:Error selecting ADF a0000000871002! Failed with SW match failed! Expected 9000 and got 6a82.
DEBUG:root:Sending TPDU: 01A4040C10A0000005591010FFFFFFFF8900000100
DEBUG:root:Received Data: None, SW: 6121
DEBUG:root:Sending TPDU: 01C0000021
DEBUG:root:Received Data: 6f1f8410a0000005591010ffffffff8900000100a5049f6501ffe0058203020200, SW: 9000
DE

['ISDR', 'ECASD', 'ESTK_FWUPD']


DEBUG:root:Disconnected from device HID Global OMNIKEY 3x21 Smart Card Reader [OMNIKEY 3x21 Smart Card Reader] 00 00


CardConnectionException: Failed to transmit with protocol T0. Transaction failed.: Transaction failed. (0x80100016)

In [1]:
from pprint import pprint
import logging

from resimulate.euicc.card import Card
from resimulate.euicc.recorder.recorder import OperationRecorder
from resimulate.euicc.transport.pcsc_link import PcscLink

logging.basicConfig(level=logging.DEBUG)

pprint(
    OperationRecorder.compare_files(
        "recordings/5ber_EuiccInfoScenario_DeterministicMutationEngine.resim",
        [
            "recordings/9esim_EuiccInfoScenario_DeterministicMutationEngine.resim",
            "recordings/xesim_EuiccInfoScenario_DeterministicMutationEngine.resim",
            "recordings/esim-me_EuiccInfoScenario_DeterministicMutationEngine.resim",
        ],
    )
)

DEBUG:root:Loaded compare recording: recordings/9esim_EuiccInfoScenario_DeterministicMutationEngine.resim
DEBUG:root:Loaded compare recording: recordings/xesim_EuiccInfoScenario_DeterministicMutationEngine.resim
DEBUG:root:Loaded compare recording: recordings/esim-me_EuiccInfoScenario_DeterministicMutationEngine.resim
DEBUG:root:Comparing main recordings/5ber_EuiccInfoScenario_DeterministicMutationEngine.resim with recordings/9esim_EuiccInfoScenario_DeterministicMutationEngine.resim
DEBUG:root:Comparing main recordings/5ber_EuiccInfoScenario_DeterministicMutationEngine.resim with recordings/xesim_EuiccInfoScenario_DeterministicMutationEngine.resim
DEBUG:root:No differences found between recordings/5ber_EuiccInfoScenario_DeterministicMutationEngine.resim and recordings/xesim_EuiccInfoScenario_DeterministicMutationEngine.resim
DEBUG:root:Comparing main recordings/5ber_EuiccInfoScenario_DeterministicMutationEngine.resim with recordings/esim-me_EuiccInfoScenario_DeterministicMutationEngine

{('recordings/5ber_EuiccInfoScenario_DeterministicMutationEngine.resim', 'recordings/9esim_EuiccInfoScenario_DeterministicMutationEngine.resim'): {'children': {('get_euicc_info_2', 'BITFLIP'): {'compare': ('6982',
                                                                                                                                                                                                              'ApduException'),
                                                                                                                                                                                                  'main': ('6d00',
                                                                                                                                                                                                           'ApduException')},
                                                                                                                                 

In [1]:
import logging
from pprint import pprint
from resimulate.euicc.card import Card
from resimulate.euicc.models.info import EuiccInfo2
from resimulate.euicc.transport.pcsc_link import PcscLink

logging.basicConfig(level=logging.DEBUG)

with PcscLink() as link:
    card = Card(link)
    euicc_info = card.isd_r.get_euicc_info_2()
    pprint(euicc_info)

DEBUG:root:Disconnected from device HID Global OMNIKEY 3x21 Smart Card Reader [OMNIKEY 3x21 Smart Card Reader] 00 00
DEBUG:root:Connecting to device HID Global OMNIKEY 3x21 Smart Card Reader [OMNIKEY 3x21 Smart Card Reader] 00 00 using protocol T1
DEBUG:root:Connected to device HID Global OMNIKEY 3x21 Smart Card Reader [OMNIKEY 3x21 Smart Card Reader] 00 00
DEBUG:root:Sending TPDU: 01A4040C07A0000000871002
DEBUG:root:Received Data: None, SW: 9000
DEBUG:root:Selected ADF a0000000871002
DEBUG:root:Found USIM via ADF a0000000871002
DEBUG:root:Sending TPDU: 01A4040C10A0000005591010FFFFFFFF8900000100
DEBUG:root:Received Data: None, SW: 6121
DEBUG:root:Sending TPDU: 01C0000021
DEBUG:root:Received Data: 6f1f8410a0000005591010ffffffff8900000100a5049f6501ffe0058203020300, SW: 9000
DEBUG:root:Selected ADF A0000005591010FFFFFFFF8900000100
DEBUG:root:Found ISDR via ADF A0000005591010FFFFFFFF8900000100
DEBUG:root:Sending TPDU: 01A4040C10A0000005591010FFFFFFFF8900000200
DEBUG:root:Received Data: Non

EuiccInfo2(base_profile_package_version='2.3.1', lowest_svn='2.3.0', euicc_firmware_version='36.17.4', ext_card_resource='81010b820400174e2083027fe3', uicc_capability=<UICCCapability flags=['USIM_SUPPORT', 'ISIM_SUPPORT', 'CSIM_SUPPORT', 'AKA_MILENAGE', 'AKA_CAVE', 'AKA_TUAK128', 'AKA_TUAK256', 'GBA_AUTHEN_USIM', 'GBA_AUTHEN_ISIM', 'EAP_CLIENT', 'JAVACARD', 'MULTIPLE_USIM_SUPPORT', 'MULTIPLE_ISIM_SUPPORT', 'MULTIPLE_CSIM_SUPPORT', 'BER_TLV_FILE_SUPPORT', 'CAT_TP', 'GET_IDENTITY', 'PROFILE_A_X25519', 'PROFILE_B_P256', 'SUCI_CALCULATOR_API']>, ts102241_version='15.1.0', globalplatform_version='2.3.0', euicc_rsp_capability=<EuiccRspCapability flags=['ADDITIONAL_PROFILE', 'TEST_PROFILE_SUPPORT', 'DEVICE_INFO_EXTENSIBILITY_SUPPORT']>, euicc_ci_pkid_list_for_verification=['81370f5125d0b1d408d4c3b232e6d25e795bebfb'], euicc_ci_pkid_list_for_signing=['81370f5125d0b1d408d4c3b232e6d25e795bebfb'], euicc_category=None, forbidden_profile_policy_rules=None, pp_version='1.0.0', sas_accreditation_numbe

In [ ]:
import logging

from resimulate.fuzzing.apdu_fuzzing import SCENARIOS
from resimulate.fuzzing.apdu_fuzzing.euicc_info import EuiccInfoScenario
from resimulate.fuzzing.apdu_fuzzing.models.scenario_runner import ScenarioRunner
from resimulate.euicc.mutation.deterministic_engine import DeterministicMutationEngine


logging.basicConfig(level=logging.DEBUG)

mutation_engine = DeterministicMutationEngine()
scenarios = [EuiccInfoScenario]

runner = ScenarioRunner(scenarios=scenarios)
runner.record_card(
    card_name="esim-me",
    mutation_engine=mutation_engine,
    path="recordings/",
    overwrite=True,
)

DEBUG:root:Cleared all recorded operations
DEBUG:root:Initializing recorder...
DEBUG:root:Disconnected from device HID Global OMNIKEY 3x21 Smart Card Reader [OMNIKEY 3x21 Smart Card Reader] 00 00
DEBUG:root:Connecting to device HID Global OMNIKEY 3x21 Smart Card Reader [OMNIKEY 3x21 Smart Card Reader] 00 00 using protocol T1
DEBUG:root:Connected to device HID Global OMNIKEY 3x21 Smart Card Reader [OMNIKEY 3x21 Smart Card Reader] 00 00
DEBUG:root:Disconnected from device HID Global OMNIKEY 3x21 Smart Card Reader [OMNIKEY 3x21 Smart Card Reader] 00 00
DEBUG:root:Disconnected from device HID Global OMNIKEY 3x21 Smart Card Reader [OMNIKEY 3x21 Smart Card Reader] 00 00
DEBUG:root:Connecting to device HID Global OMNIKEY 3x21 Smart Card Reader [OMNIKEY 3x21 Smart Card Reader] 00 00 using protocol T1
DEBUG:root:Connected to device HID Global OMNIKEY 3x21 Smart Card Reader [OMNIKEY 3x21 Smart Card Reader] 00 00
DEBUG:root:Sending TPDU: 01A4040C07A0000000871002
DEBUG:root:Received Data: None, SW